# Agent智能代理

## 一、Agents 基础认知：什么是智能代理？

在Langchain中，Agent是一类能自主决策、调用工具解决复杂问题的模块 ———— 它不像 Chains 那样按固定流程执行，而是以LLM为“大脑”，根据用户需求动态判断：

- 是否需要调用工具（如搜索、计算、数据库查询）

- 调用哪个工具、传递什么参数

- 如何利用工具返回结果生成最终回答

- 是否需要多轮工具调用

其核心逻辑是“根据观察到的环境 > 做决策 > 将决策结果与旧环境生成新环境 > 做决策”循环。


## 二、Agents与Chains的核心区别

|维度|Chains|Agents|
|--|--|--|
|执行逻辑|固定流程|根据观测到的环境，动态决策|
|工具依赖|需手动指定工具调用时机和参数|自动选择工具、生成参数|
|适用场景|流程固定的任务|需外部信息/动态判断的任务|
|灵活性|低|高|

## 三、Agents核心组件

要构建一个可用的Agent，需4个核心部分：

1. **LLM（决策大脑）**：负责根据环境做出合理的决策

2. **工具（Tool）**：Agent做决策的工具

3. **记忆（Memory）**：保存历史观测的环境和决策结果，帮助LLM更好的做决策。

4. **执行器（Agent Executor）**：管理“思考 > 行动 > 观察”循环，确保工具调用和结果处理正常。

## 四、实践案例：从基础到进阶搭建Agent

### 前期准备

1. **安装依赖**

In [ ]:
# 核心依赖：LangGraph（Agent框架）、Tavily搜索工具、内存组件
!pip install -U langgraph langchain-tavily

2. **获取API密钥**

    - **Tavily API**：免费搜索工具，注册地址[https://tavily.com](https://tavily.com)，获取后保存；


### 案例1：基础搜索Agent（无记忆，单轮交互）

目标：构建一个能调用Tavily搜索的Agent，回答“实时天气”这类需要外部信息的问题。

In [6]:
from dotenv import load_dotenv
import os
import getpass
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent # ReAct Agent执行器

load_dotenv()

# 1. 设置API密钥
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("输入Tavily API密钥：")
else:
    print(f"Tavily 密钥加载成功：{os.getenv("TAVILY_API_KEY")[:6]}")

llm = ChatOpenAI(
    openai_api_key=os.getenv("UIUIAPI_API_KEY"),
    base_url=os.getenv("UIUIAPI_BASE_URL"),
    model="gpt-3.5-turbo",
    temperature=0.1 # 决策任务用低温度，避免随机性
)

search_tool = TavilySearch(max_results=2)
tools = [search_tool] # Agent可调用的工具集

# 2. 创建Agent执行器
agent_executor = create_react_agent(
    model=llm,
    tools=tools,
)

# 3. 运行Agent
input_msg = {"role": "user", "content": "江苏苏州市现在的天气怎么样？"}

response = agent_executor.invoke({'messages': [input_msg]})

for msg in response["messages"]:
    msg.pretty_print()


Tavily 密钥加载成功：tvly-d
================================ Human Message =================================

江苏苏州市现在的天气怎么样？
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_Rc2NdhNBTJBqpC2Liidlo7vh)
 Call ID: call_Rc2NdhNBTJBqpC2Liidlo7vh
  Args:
    query: 江苏苏州市现在的天气
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "江苏苏州市现在的天气", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://tianqi.moji.com/today/china/jiangsu/suzhou", "title": "今天苏州市天气_今日天气预报- 墨迹天气", "content": "苏州市今天实况：28度多云，湿度：87%，南风：2级。白天：36度,晴。 夜间：多云，27度，潮湿闷热，墨迹天气建议您穿着吸湿排汗的夏装，棉麻质地的短打扮是不错的选择。", "score": 0.829285, "raw_content": null}, {"url": "https://tianqi.moji.com/weather/china/jiangsu/suzhou", "title": "苏州市天气预报_天气查询- 墨迹天气", "content": "苏州市今天实况：27度雾，湿度：90%，东北风：1级。白天：36度,小雨。 夜间：多云，27度，潮湿闷热，墨迹天气建议您穿着吸湿排汗的夏装，棉麻质地的短打扮是不错的选择。", "score": 0.82895297,

### 案例2：带记忆的Agent

目标：让Agent记住对话历史（如用户先介绍“我住在江苏苏州市”，后续问“我居住地的天气”时，Agent能关联上下文，无需重复说明地点）。

In [11]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

load_dotenv()

llm = ChatOpenAI(
    openai_api_key=os.getenv("UIUIAPI_API_KEY"),
    base_url=os.getenv("UIUIAPI_BASE_URL"),
    model="gpt-3.5-turbo",
    temperature=0.1
)

search_tool = TavilySearch(max_results=2)
tools = [search_tool]

# 初始化内存（保存对话历史，基于thread_id区分不同对话）
memory = MemorySaver()

# 创建带记忆的Agent执行器
agent_executor = create_react_agent(
    model=llm,
    tools=tools,
    checkpointer=memory # 传入内存组件
)

# 配置对话ID
config = {"configurable": {"thread_id": "my_chat_001"}}

# 第一轮：自我介绍
input1 = {"role":"user", "content": "Hi, 我是LCY，我住在江苏苏州市"}
print("=== 第一轮交互 ===")

for step in agent_executor.stream(
    {"messages": [input1]},
    config,
    stream_mode="values" # 流式输出每一步结果
):
    step["messages"][-1].pretty_print()

# 第二轮：用户问“我那的天气”（不重复提地点，测试记忆）
input2 = {"role": "user", "content": "我住的地方现在天气怎么样？"}
print("\n=== 第二轮交互 ===")
for step in agent_executor.stream(
    {"messages": [input2]}, 
    config, 
    stream_mode="values"
):
    step["messages"][-1].pretty_print()


=== 第一轮交互 ===
================================ Human Message =================================

Hi, 我是LCY，我住在江苏苏州市
================================== Ai Message ==================================

你好，LCY！有什么可以帮助你的吗？

=== 第二轮交互 ===
================================ Human Message =================================

我住的地方现在天气怎么样？
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_2TCjM8FFh4ahxENdVFRlacaf)
 Call ID: call_2TCjM8FFh4ahxENdVFRlacaf
  Args:
    query: 苏州市天气现况
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "苏州市天气现况", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.weather.com.cn/weather/101190401.shtml", "title": "预报- 苏州 - 中国天气网", "content": "首页 预报 预警 雷达 云图 天气地图 专业产品 资讯 视频 节气 我的天空 全国> 江苏 苏州> 城区 11:30更新 | 数据来源 中央气象台 * 今天 * 7天 * 8-15天 * 40天 * 雷达图 * # 24日（今天） 37/*29℃* *<3级* * # 25日（明天） 37/*29℃*

### 案例3：流式输出Agent（实时展示思考/工具过程）

In [25]:
import time

ask_content="苏州未来2天会下雨吗?"
input_msg = {"role": "user", "content": ask_content}

for step, metadata in agent_executor.stream(
    {"messages": [input_msg]},
    config, # 复用之前的对话ID
    stream_mode="messages", # 流式输出每一条消息
):
    for char in step.content:
        print(char, end='', flush=True)
        time.sleep(0.01)  # 控制打字速度

根据最新的天气预报，苏州市未来两天的天气情况如下：

- **明天（25日）**：预计天气晴，最高温度37℃，最低温度29℃，没有降水。
- **后天（26日）**：预计有雷阵雨转多云，最高温度36℃，最低温度28℃，可能会有降水。

因此，后天会有降雨的可能性。

关键说明

- `stream_mode="values"`：输出完整的 “步骤结果”（如整段 AI 回答、完整工具结果）；

- `stream_mode="messages"`：输出 “逐 token” 的实时内容（如 AI 回答时一个字一个字打印）；

- 适合构建聊天机器人等交互场景，让用户知道 Agent “正在工作”，而非等待空白。

## 五、自定义工具：拓展Agent能力

目标：给Agent添加计算器工具，解决数学计算问题

In [27]:
from langchain_core.tools import tool # 自定义工具装饰器

def calculator(a: int, b: int, op: str) -> int:
    """
    用于计算两个整数的四则运算（加、减、乘、除）。
    参数：
        a: 第一个整数
        b: 第二个整数
        op: 运算符（+、-、*、/，除法结果取整）
    返回：
        计算结果
    """
    if op == "+":
        return a + b
    elif op == "-":
        return a - b
    elif op == "*":
        return a * b
    elif op == "/":
        return a // b  # 整数除法
    else:
        raise ValueError("运算符仅支持+、-、*、/")

tools_with_calc = [search_tool, calculator]

agent = create_react_agent(
    model=llm,
    tools=tools_with_calc
)

msg = {"role": "user", "content": "123乘以456等于多少？"}
response = agent.invoke({"messages": [msg]})

for msg in response["messages"]:
    msg.pretty_print()


================================ Human Message =================================

123乘以456等于多少？
================================== Ai Message ==================================
Tool Calls:
  calculator (call_nhz74F6ofdco75Rj5xVzeRZ6)
 Call ID: call_nhz74F6ofdco75Rj5xVzeRZ6
  Args:
    a: 123
    b: 456
    op: *
================================= Tool Message =================================
Name: calculator

56088
================================== Ai Message ==================================

123乘以456等于56088。


**关键说明**

- 自定义工具需用 `@tool`装饰器， 且必须写清晰的“工具描述” ———— LLM会根据描述判断是否调用该工具。

- 工具参数需明确类型（如`a:int`），帮助LLM正确生成参数

- 可拓展更多工具（如数据库查询、API调用），让Agent支持更复杂的场景。
